# 1.0. Import

In [7]:
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RationalQuadratic
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import tensorflow as tf

In [8]:
#Importando o DataSet
df_train = pd.read_csv("./Bancos_utilizados/Bancos_apos_filtragem/train.csv")
df_valid = pd.read_csv("./Bancos_utilizados/Bancos_apos_filtragem/valid.csv")

# 2.0. Validar Novo Banco de Dados

Espaço para testar o desempenho do modelo em outro banco de dados buscado na literatura.

## 2.1. Preparando para testar os Modelos

Primeiro vamos criar um DataSet para armazenar o resultado de cada modelo.

In [9]:
colunas = ['MODEL','R2','RMSE','MAE','MAE_MAX','MAE_MIN']
Metrics = pd.DataFrame(columns = colunas)

Vamos também já organizar o treino como o dataset antigo e o teste como o novo, já aplicando o StandardScaler para alterar a escala.

In [10]:
scaler = StandardScaler()

X_treino = df_train.drop(['Resistência'], axis=1)
X_treino = scaler.fit_transform(X_treino)
y_treino = df_train['Resistência']

X_valid = df_valid.drop(['Resistência'], axis=1)
X_valid = scaler.transform(X_valid)
y_valid = df_valid['Resistência']

Vamos ainda criar uma função para calcular metricas:

In [11]:
def calcular_metricas(y_true, y_pred, name_model):
    MAE = mean_absolute_error(y_true, y_pred)              
    RMSE = mean_squared_error(y_true, y_pred, squared=False)    
    R2 = r2_score(y_true, y_pred) 
    MAE_MAX = abs(y_pred-y_true).max()
    MAE_MIN = abs(y_pred-y_true).min()

    return {'MODEL': name_model,'MAE': MAE,'RMSE': RMSE, 'R2': R2,
                        'MAE_MAX': MAE_MAX,'MAE_MIN': MAE_MIN}  

## 2.2. Modelos

### 2.2.1. Decision Tree

In [30]:
xgbc = xgb.XGBRegressor(colsample_bytree= 0.898275838301962, 
                            learning_rate= 0.10335797738188568,
                            max_depth = 10,
                            min_child_weight= 3.5585146125243106, 
                            subsample= 0.7384101157071529)
xgbc.fit(X_treino, y_treino)
pred_xgb = xgbc.predict(X_valid)

Metrics = Metrics.append(calcular_metricas(y_valid,pred_xgb,'XGBoost'), ignore_index=True)

### 2.2.2. SVR

In [12]:
svr_model = SVR(kernel='rbf', C=32.919679770421595, gamma = 0.6017663761743526)
svr_model.fit(X_treino, y_treino)
pred_svr = svr_model.predict(X_valid)

Metrics = Metrics.append(calcular_metricas(y_valid,pred_svr,'SVR'), ignore_index=True)

### 2.2.3. RNA

In [32]:
#Função para obter resultados reprodutiveis 
tf.random.set_seed(1)

# Vamos definir EarlyStopping. Isso faz com que o modelo pare quando parar de melhorar as predições
early_stopping = callbacks.EarlyStopping(
    min_delta=0.05,
    patience=15,
    restore_best_weights=True)

#Definimos o Modelo
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[X_treino.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1),
])


model.compile(
    optimizer='sgd', 
    loss='mae',
    metrics=['mae','mse'],
)
history = model.fit(
    X_treino, y_treino,
    validation_split=0.1, #dados de teste
    batch_size=45,
    epochs=150,
    verbose=0,
    callbacks=[early_stopping],

)

rna_pred = model.predict(X_valid)
rna_pred_reshape  = rna_pred.reshape((23,))

Metrics = Metrics.append(calcular_metricas(y_valid,rna_pred_reshape,'RNA'), ignore_index=True)

### 2.2.4. Gaussian

In [33]:
# Modelo: Definimos o Kernel, Treinamos e fazemos a predição
kernel = ConstantKernel() * RationalQuadratic()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.23604724748727526,random_state=0).fit(X_treino, y_treino)
pred_gpr = gpr.predict(X_valid)

Metrics = Metrics.append(calcular_metricas(y_valid,pred_gpr,'GPR'), ignore_index=True)

## 2.3. Resultado

In [13]:
Metrics

,MODEL,R2,RMSE,MAE,MAE_MAX,MAE_MIN
0,SVR,-0.035628,5.967456,5.1416,12.804034,0.002165
